In [ ]:
%%writefile requirements.txt
python-dotenv==0.20.0
mysql-connector-python==8.0.30
ipython-sql==0.4.1

In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import mysql.connector
import pandas as pd
import sqlalchemy

In [ ]:
%%writefile .env
HOST1=
USER1=
PASSWORD1=
DATABASE1=

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
USERNAME = os.getenv('USER1')
PASSWORD = os.getenv('PASSWORD1')
HOST = os.getenv('HOST1')
DATABASE = os.getenv('DATABASE1')
print(USERNAME , HOST, DATABASE)

In [ ]:
TABLE = "us_housing_units"

In [13]:
conn_str = 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(USERNAME, PASSWORD, HOST, DATABASE)
conn = sqlalchemy.create_engine(conn_str)

In [14]:
%config SqlMagic.autopandas=True
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False
%reload_ext sql
%sql {conn_str}

In [ ]:
!aws s3 ls s3://jan16-data/projects/sql/

In [ ]:
!mkdir -p data
!aws s3 cp s3://jan16-data/projects/sql/{TABLE}.csv data/{TABLE}.csv

In [16]:
df = pd.read_csv(f"data/{TABLE}.csv")
df

year  month month_name  south  west  midwest  northeast
0    1968.0    1.0    January   35.6  17.0     22.6       12.9
1    1968.0    2.0   February   31.5  18.6     23.3        9.7
2    1968.0    3.0      March   42.5  17.4     24.4       10.7
3    1968.0    4.0      April   42.9  20.6     27.0       12.0
4    1968.0    5.0        May   46.2  20.0     25.1       20.0
..      ...    ...        ...    ...   ...      ...        ...
554  2014.0    3.0      March   37.7  15.8      9.8        4.4
555  2014.0    4.0      April   34.2  15.0      9.2        5.5
556  2014.0    5.0        May   33.7  17.5     12.2       10.0
557  2014.0    6.0       June   36.2  15.2     13.1        7.0
558  2014.0    7.0       July   35.8  15.9     12.9        5.9

[559 rows x 7 columns]

In [ ]:
df.info()

In [ ]:
df["end_time"] = pd.to_datetime(df["end_time"])
df.info()

In [ ]:
df.to_csv(f"data/{TABLE}.csv", index=False)

In [17]:
print(pd.io.sql.get_schema(df,
                           name=TABLE,
                           con=conn))


CREATE TABLE us_housing_units (
	year FLOAT(53), 
	month FLOAT(53), 
	month_name TEXT, 
	south FLOAT(53), 
	west FLOAT(53), 
	midwest FLOAT(53), 
	northeast FLOAT(53)
)




In [18]:
%%sql

CREATE TABLE IF NOT EXISTS us_housing_units (
	year FLOAT(53), 
	month FLOAT(53), 
	month_name TEXT, 
	south FLOAT(53), 
	west FLOAT(53), 
	midwest FLOAT(53), 
	northeast FLOAT(53)
)

Empty DataFrame
Columns: []
Index: []

In [ ]:
%%sql
truncate table {TABLE}

Mac/Linux

In [ ]:
!echo mysqlimport --local \
    --compress \
    --user={USERNAME} \
    --password \
    --host={HOST} \
    --ignore-lines=1 \
    --fields-terminated-by=\',\' {DATABASE} data/{TABLE}.csv 

Windows

In [ ]:
!echo mysqlimport --local \
    --compress \
    --user={USERNAME} \
    --password \
    --host={HOST} \
    --ignore-lines=1 \
    --fields-terminated-by=',' {DATABASE} data\{TABLE}.csv 

In [ ]:
%%sql
select count(*) from {TABLE}

In [ ]:
%%sql
select * from {TABLE} limit 10